In [1]:
!pip install tensorflow==1.13.1

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/gdrive')
gitDir = "/content/gdrive/My Drive/nlp/"
os.chdir(gitDir + "data/")
print(os.listdir("."))

#sys.path.insert(0,gitDir + "data")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
['Kaggle Amazon Fine Food.ipynb', '.ipynb_checkpoints', 'Organic Dataset.ipynb', 'cache', '__pycache__', 'Amazon Reviews.ipynb', 'Amazon Multilingual.ipynb', 'tmpBACKUP.py', 'tmp.ipynb', 'amazon_multilingual.py', 'Dataset_Amazon_Multilingual.ipynb']


In [0]:
import tensorflow as tf
import os

import shutil, gzip
import pandas as pd

import numpy as np

In [0]:
def getCategory2IndexDict():
    d = {'Video DVD':                   0,
         'Music':                       1,
         'Books':                       2,
         'Mobile_Apps':                 3,
         'Digital_Video_Download':      4,
         'Digital_Music_Purchase':      5,
         'Toys':                        6,
         'Digital_Ebook_Purchase':      7,
         'PC':                          8,
         'Camera':                      9,
         'Wireless':                    10,
         'Electronics':                 11,
         'Video':                       12,
         'Sports':                      13,
         'Video Games':                 14,
         'Watches':                     15,
         'Shoes':                       16,
         'Home':                        17,
         'Musical Instruments':         18,
         'Baby':                        19,
         'Home Improvement':            20,
         'Home Entertainment':          21,
         'Office Products':             22,
         'Personal_Care_Appliances':    23,
         'Automotive':                  24,
         'Lawn and Garden':             25,
         'Luggage':                     26,
         'Kitchen':                     27,
         'Furniture':                   28,
         'Health & Personal Care':      29,
         'Software':                    30,
         'Grocery':                     31,
         'Pet Products':                32,
         'Beauty':                      33,

         #UK
         'Apparel':                     34,

         #US
         'Tools':                       35,
         'Outdoors':                    36,
         'Mobile_Electronics':          37,
         '2012-12-22':                  38,
        }
    return d

def generateFilenames(countryCode):
    x1 = "cache/amazon_multilingual_"+countryCode+"_category.npy"
    x2 = "cache/amazon_multilingual_"+countryCode+"_labels.npy"
    return (x1,x2)

def getAmazonDataFrame(filename):
    p = tf.keras.utils.get_file(filename,origin="https://s3.amazonaws.com/amazon-reviews-pds/tsv/" + filename,
                                extract=False, cache_dir=".", cache_subdir="cache")
    outDir = p[0:-3]
    if os.path.exists(outDir) == False:
        with open(outDir, 'wb') as f_out, gzip.open(p, 'rb') as f_in:
            shutil.copyfileobj(f_in, f_out)
    else:
        print(outDir, "already exists. Skipping unzipping")

    frame = pd.read_csv(p[0:-3], sep='\t', error_bad_lines=False)
    return frame


def dataToFile(data, countryCode):
    f1, f2 = generateFilenames(countryCode)
    d = getCategory2IndexDict()

    # TODO: also use headline (review_headline) ?
    x = data.review_body.values
    y = [d[x] for x in data.product_category.values]

    np.save(f1, x)
    np.save(f2, y)


In [0]:
def getData(countryCode, shuffle, buffer=None, batchsize=128):  # DE, UK, US

    f1, f2 = generateFilenames(countryCode)
    if os.path.exists(f1) and os.path.exists(f2):
        print(f1, "and", f2, " exist. Using saved npy files...")
    else:
        print(f1, "or", f2, " missing. Creating it...")
        frame = getAmazonDataFrame("amazon_reviews_multilingual_" + countryCode + "_v1_00.tsv.gz")
        print(frame.shape)
        frame.dropna( subset=['review_body'], inplace=True )
        print(frame.shape)
        dataToFile(frame, countryCode) #TODO!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    x = np.load(f1, allow_pickle=True)
    y = np.load(f2)

    #print(x.shape, y.shape)
    #print(x[:4])
    #print(y[:4])
    #print(type(y[0]))
    return buildDataset(x, y, shuffle, buffer, batchsize)



def buildDataset(x, y, shuffle, buffer, batchsize):
    length = x.shape[0]

    if buffer is None:
        buffer = length
        
    print(x.shape, y.shape)
    print(type(x), type(y))
    
    
    features_placeholder = tf.placeholder(x.dtype, x.shape)
    labels_placeholder = tf.placeholder(y.dtype, y.shape)


    dataset = tf.data.Dataset.from_tensor_slices( (features_placeholder, labels_placeholder) )
    if shuffle == True:
        # tf.data.experimental.AUTOTUNE
        dataset = dataset.shuffle(buffer).batch(batchsize).prefetch(1)
    else:
        print("dataset is not shuffled and prefetched")

    iterator = dataset.make_initializable_iterator()
    feed_dict={features_placeholder: x, labels_placeholder: y}
    
    '''
    with tf.Session() as sess:
      sess.run(iterator.initializer, feed_dict=feed_dict)
      output = sess.run(iterator.get_next())
      print("out:", output)
    '''

    return iterator, feed_dict, length

In [28]:
iterator, feed_dict, length = getData("UK", shuffle=True, buffer=4, batchsize=4)

cache/amazon_multilingual_UK_category.npy or cache/amazon_multilingual_UK_labels.npy  missing. Creating it...
./cache/amazon_reviews_multilingual_UK_v1_00.tsv already exists. Skipping unzipping


b'Skipping line 101882: expected 15 fields, saw 22\nSkipping line 115512: expected 15 fields, saw 22\n'
b'Skipping line 328404: expected 15 fields, saw 22\n'
b'Skipping line 1412642: expected 15 fields, saw 22\n'


(1705837, 15)
(1705822, 15)
(1705822,) (1705822,)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [25]:
with tf.Session() as sess:
  
  sess.run(iterator.initializer, feed_dict=feed_dict)
  
  output = sess.run(iterator.get_next())
  print("out:", output)

out: (array([b'What a great watch. Both watches and strap is in a great quality, and the prize is low. Especially compared to the price here in Denmark.',
       b"love watching all the episodes of Dexter, when i first heard about this series i wasnt too sure about watching it. it took me a very long time to start and now wish i had watched it when it first came out.<br /><br />The fact that he kills and more or less gets away with it, in some episodes you sat on edge of your seat thinking OMG he gonna get caught and then he doesn't, he does try stopping but you know yourself he can't. my favorite episode where the Ice Truck killer, that the ending of that was great how he learns he had a brother, but for his brother to be a baddie and that he had to kill him, i think if he hadn't would it have been different. Oh and The Bay Harbour Butcher, in this it made you think oh no he gonna get caught but he didn't<br /><br />In all ive enjoyed watching them and a shame he had to end.",
       

In [27]:
!rm cache/amazon_multilingual_UK*

rm: cannot remove 'cache/amazon_multilingual_UK*': No such file or directory


In [29]:
!ls -alh cache

total 12G
-rw------- 1 root root 473M May 14 16:50 amazon_multilingual_DE_category.npy
-rw------- 1 root root 2.6M May 14 16:50 amazon_multilingual_DE_labels.npy
-rw------- 1 root root 609M May 14 18:59 amazon_multilingual_UK_category.npy
-rw------- 1 root root  14M May 14 18:59 amazon_multilingual_UK_labels.npy
-rw------- 1 root root 2.6G May 14 18:22 amazon_multilingual_US_category.npy
-rw------- 1 root root  53M May 14 18:22 amazon_multilingual_US_labels.npy
-rw------- 1 root root 563M May 13 16:19 amazon_reviews_multilingual_DE_v1_00.tsv
-rw------- 1 root root 231M May 13 16:18 amazon_reviews_multilingual_DE_v1_00.tsv.gz
-rw------- 1 root root 821M May 14 14:03 amazon_reviews_multilingual_UK_v1_00.tsv
-rw------- 1 root root 334M May 13 16:35 amazon_reviews_multilingual_UK_v1_00.tsv.gz
-rw------- 1 root root 3.4G May 13 19:12 amazon_reviews_multilingual_US_v1_00.tsv
-rw------- 1 root root 1.4G May 13 16:27 amazon_reviews_multilingual_US_v1_00.tsv.gz
-rw------- 1 root root 9.1M May 1

## tmp experiments

In [0]:
frame = getAmazonDataFrame("amazon_reviews_multilingual_" + "UK" + "_v1_00.tsv.gz")
frame.head()

./cache/amazon_reviews_multilingual_UK_v1_00.tsv already exists. Skipping unzipping


b'Skipping line 101882: expected 15 fields, saw 22\nSkipping line 115512: expected 15 fields, saw 22\n'
b'Skipping line 328404: expected 15 fields, saw 22\n'
b'Skipping line 1412642: expected 15 fields, saw 22\n'


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,UK,10349,R2YVNBBMXD8KVJ,B00MWK7BWG,307651059,My Favourite Faded Fantasy,Music,5,0,0,N,Y,Five Stars,The best album ever!,2014-12-29
1,UK,10629,R2K4BOL8MN1TTY,B006CHML4I,835010224,Seiko 5 Men's Automatic Watch with Black Dial ...,Watches,4,0,0,N,Y,Great watch from casio.,What a great watch. Both watches and strap is ...,2013-10-24
2,UK,12136,R3P40IEALROVCH,B00IIFCJX0,271687675,Dexter Season 8,Digital_Video_Download,5,0,0,N,Y,fantastic,"love watching all the episodes of Dexter, when...",2014-05-09
3,UK,12268,R25XL1WWYRDLA9,B000W7JWUA,211383699,The Settlers of Catan Board Game - discontinue...,Toys,5,0,0,N,Y,Five Stars,Excellent game!!!,2014-09-19
4,UK,12677,RVTVB9YDXSFYH,B005JTAP4S,182965893,Peter: A Darkened Fairytale (Vol 1),Digital_Ebook_Purchase,5,12,12,N,N,A twist on Tales,"This cute, quick read is very different to say...",2013-09-18


In [0]:
frame.product_category.value_counts()

Mobile_Apps                 1467128
Digital_Ebook_Purchase      1242173
Video DVD                   1093612
Digital_Video_Download      1051622
Books                        836136
Music                        776810
Digital_Music_Purchase       107461
Toys                          57465
PC                            56817
Video                         46697
Home Entertainment            36298
Wireless                      22637
Camera                        16911
Video Games                   15398
Electronics                   11421
Musical Instruments           10914
Watches                       10537
Tools                          7475
Shoes                          7342
Baby                           5868
Sports                         4123
Home Improvement               3698
Outdoors                       3184
Office Products                2306
Home                           1998
Kitchen                        1834
Lawn and Garden                1204
Health & Personal Care      

In [0]:
print(frame.shape)
cut = frame.dropna( subset=['review_body'] )
print(cut.shape)
cut.head()

(1705837, 15)
(1705822, 15)


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,UK,10349,R2YVNBBMXD8KVJ,B00MWK7BWG,307651059,My Favourite Faded Fantasy,Music,5,0,0,N,Y,Five Stars,The best album ever!,2014-12-29
1,UK,10629,R2K4BOL8MN1TTY,B006CHML4I,835010224,Seiko 5 Men's Automatic Watch with Black Dial ...,Watches,4,0,0,N,Y,Great watch from casio.,What a great watch. Both watches and strap is ...,2013-10-24
2,UK,12136,R3P40IEALROVCH,B00IIFCJX0,271687675,Dexter Season 8,Digital_Video_Download,5,0,0,N,Y,fantastic,"love watching all the episodes of Dexter, when...",2014-05-09
3,UK,12268,R25XL1WWYRDLA9,B000W7JWUA,211383699,The Settlers of Catan Board Game - discontinue...,Toys,5,0,0,N,Y,Five Stars,Excellent game!!!,2014-09-19
4,UK,12677,RVTVB9YDXSFYH,B005JTAP4S,182965893,Peter: A Darkened Fairytale (Vol 1),Digital_Ebook_Purchase,5,12,12,N,N,A twist on Tales,"This cute, quick read is very different to say...",2013-09-18
